<a href="https://colab.research.google.com/github/nisbenz/Mini-Transformer/blob/main/InputProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files

uploaded = files.upload()

Saving simplemath_100k_balanced.csv to simplemath_100k_balanced.csv


In [7]:
import pandas as pd
import io
file_name = 'simplemath_100k_balanced.csv'
ds = pd.read_csv(file_name)
print(f"Successfully loaded '{file_name}' into a DataFrame.")
display(ds.head())

Successfully loaded 'simplemath_100k_balanced.csv' into a DataFrame.


,problem,answer
0,2790 + 6698 =,9488
1,8262 + 3848 =,12110
2,1628 + 132 =,1760
3,9925 + 7169 =,17094
4,3023 + 6929 =,9952


In [8]:
import pandas as pd
X_train = pd.Series(ds["problem"])
X_train = X_train.str.ljust(13, ' ')
raw_chars = sorted(list(set(''.join(X_train))))
vocab =  raw_chars
stoi = { ch:i for i,ch in enumerate(vocab) }
itos = { i:ch for i,ch in enumerate(vocab) }
print(f"Vocabulary Size: {len(vocab)}")
print("Character Map:", stoi)
def encode_problem(problem_str):
    return [stoi[c] for c in problem_str]
X_train_encoded = [encode_problem(x) for x in X_train]
print("\nExample Output:")
print(f"Original: '{X_train[0]}'")
print(f"Encoded:  {X_train_encoded[0]}")




Vocabulary Size: 16
Character Map: {' ': 0, '+': 1, '-': 2, '/': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '=': 14, 'x': 15}

Example Output:
Original: '2790 + 6698 ='
Encoded:  [6, 11, 13, 4, 0, 1, 0, 10, 10, 13, 12, 0, 14]


In [17]:
import torch
import torch.nn as nn
import math
VOCAB_SIZE = 19
D_MODEL = 128
MAX_LEN = 128
embedding = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=D_MODEL, padding_idx=0)
X_train_tensor = torch.tensor(X_train_encoded, dtype=torch.long)
X_embedded = embedding(X_train_tensor)
print(f"Shape of X_embedded: {X_embedded.shape}")
print(f"Value of X_embedded[0][0]: {X_embedded[0][0]}")

Shape of X_embedded: torch.Size([75000, 13, 128])
Value of X_embedded[0][0]: tensor([ 2.3338,  1.4696, -0.9279,  0.6853,  0.4009,  1.5441,  0.0559, -0.6910,
         0.8127,  0.5362,  0.5750, -0.4635,  1.0020, -1.3314, -0.0759,  0.4433,
        -0.1582, -0.1827, -0.5598, -1.8547, -0.2664,  0.0124, -0.2821,  0.4072,
        -0.9772, -0.5881,  2.0837, -0.4291,  0.6630, -0.7507, -0.3861,  0.6640,
         0.5045, -0.0204,  0.3200,  0.9982,  0.0440,  1.0578, -0.4164, -0.0285,
         0.9557,  0.3958,  1.1669,  1.9372,  0.2008,  0.0467,  1.2442,  0.6110,
        -1.2195, -0.5511, -0.3169, -0.7557, -1.0515, -0.3398, -0.6129, -0.6935,
         0.6023,  0.3770, -0.2886, -0.3850, -1.6721, -0.7573, -1.6338,  0.8618,
        -0.0674,  0.1118, -0.8384,  0.6042,  0.3806,  2.2688,  0.6265,  0.5506,
        -0.1121,  2.4947, -0.4043, -0.8253, -0.2823, -0.0646,  0.3479,  0.9708,
         0.4532, -0.3512,  0.3313,  0.3593,  0.2155, -2.5051, -0.3332,  0.4102,
         0.5470,  0.5281,  2.5942,  0.4379,

In [18]:
import numpy as np
import tensorflow as tf

def positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / np.float32(d_model))
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)


seq_len = X_embedded.shape[1]
d_model = X_embedded.shape[2]

pos_enc_tf = positional_encoding(seq_len, d_model)

pos_enc_torch = torch.from_numpy(pos_enc_tf.numpy()).to(X_embedded.device)

X_final = X_embedded + pos_enc_torch

print(f"Final Input Shape: {X_final.shape}")
print(f"First value after position added: {X_final[0][0]}")


Final Input Shape: torch.Size([75000, 13, 128])
First value after position added: tensor([ 2.3338,  2.4696, -0.9279,  1.6853,  0.4009,  2.5441,  0.0559,  0.3090,
         0.8127,  1.5362,  0.5750,  0.5365,  1.0020, -0.3314, -0.0759,  1.4433,
        -0.1582,  0.8173, -0.5598, -0.8547, -0.2664,  1.0124, -0.2821,  1.4072,
        -0.9772,  0.4119,  2.0837,  0.5709,  0.6630,  0.2493, -0.3861,  1.6640,
         0.5045,  0.9796,  0.3200,  1.9982,  0.0440,  2.0578, -0.4164,  0.9715,
         0.9557,  1.3958,  1.1669,  2.9372,  0.2008,  1.0467,  1.2442,  1.6110,
        -1.2195,  0.4489, -0.3169,  0.2443, -1.0515,  0.6602, -0.6129,  0.3065,
         0.6023,  1.3770, -0.2886,  0.6150, -1.6721,  0.2427, -1.6338,  1.8618,
        -0.0674,  1.1118, -0.8384,  1.6042,  0.3806,  3.2688,  0.6265,  1.5506,
        -0.1121,  3.4947, -0.4043,  0.1747, -0.2823,  0.9354,  0.3479,  1.9708,
         0.4532,  0.6488,  0.3313,  1.3593,  0.2155, -1.5051, -0.3332,  1.4102,
         0.5470,  1.5281,  2.5942,  1.